# NTU Auto Clock-in System

### Import modules
* `selenium`: Main module for me to crawl and simulate the clock-in behavior
* `schedule`: Module that could schedule work

In [1]:
from selenium import webdriver
import time
import datetime
import schedule
from threading import Timer

#### Modules that help me to send emails
I build a auto email mechnism that send me mail every morning/evening, and the outcome of clock-in or out

In [3]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

#### Randomize the clock-in/out time

In [4]:
from random import randint

### Import Personal Information
I save my NTU username and password in the file `account.py`. Also, since I want to send myself email, I would add my email information in the same file.
* Note: "email password" here is not the one you actually use to login google. Please check this article:
https://www.learncodewithmike.com/2020/02/python-email.html

In [2]:
from account import us, ps, ml, mlps

## Functions for complete the task

In [5]:
# Check whether the login button exist, return the button element if it exist.
def find_login_button(eles):
    found_button = False
    for ele in eles:
        txt = ele.text
        if txt == '登入':
            found_button = True
            return ele
    if not found_button:
        return False

In [10]:
# Actual login function, key in username, password and click the login buttom
def login(us, ps, driver):
    time.sleep(10)
    count = 10
    while count > 0:
        try:
            eles = driver.find_elements_by_tag_name('a')
            login_button = find_login_button(eles)
            if login_button:
                login_button.click()
            else:
                print('No login button')
                time.sleep(10)
            userbox = driver.find_element_by_name('user')
            userbox.send_keys(us)
            passbox = driver.find_element_by_name('pass')
            passbox.send_keys(ps)
            driver.find_element_by_css_selector('input[type=submit]').click()
            break
        except:
            count -= 1

In [ ]:
# Check whether the clock-in or out action success, by checking the date. 
# I check the last time record(row) I could obtain.
# If the action is clock-in, I check certain column, and so as clock-out.

def check_record(tp, driver):
    if tp == 'in':
        index = 4
    else:
        index = 5
    count = 10
    while count > 0:
        try:
            records = driver.find_elements_by_tag_name('td')
            dt = [int(i) for i in records[3].text.split('-')]
            tm = [int(i) for i in records[index].text.split(':')]
            if tp == 'in':
                if (datetime.date.today() == datetime.date(dt[0],dt[1],dt[2])) and (datetime.datetime(dt[0],dt[1],dt[2],tm[0],tm[1],tm[2]) <= datetime.datetime(dt[0],dt[1],dt[2],9)):
                    return True, dt+tm
            else:
                if len(tm) > 1:
                    return True, None
        except:
            count -= 1
    return False, None

In [ ]:
# Send emails of action result. I put the text in the subject.
def send_email(s):
    content = MIMEMultipart()
    content['subject']  = s
    content['from'] = ml
    content['to'] = ml
    with smtplib.SMTP(host='smtp.gmail.com', port='587') as smtp:
        try:
            smtp.ehlo()
            smtp.starttls()
            smtp.login(ml, mlps)
            smtp.send_message(content)
        except:
            with open('error_log.txt', 'a') as f:
                print(datetime.datetime.now(), file=f)
                print('Cannot send email', file=f)

In [8]:
# Main function fo clock-in
def clock_in(us, ps):
    driver = webdriver.Chrome('C:\\Users\\Administrator\\Downloads\\chromedriver_win32\\chromedriver')
    time.sleep(10)
    # NTU account login page
    driver.get('https://my.ntu.edu.tw/attend/ssi.aspx')
    login(us, ps, driver)
    time.sleep(10)
    # Clicking clock-in buttom
    count = 10
    while count > 0:
        try:
            driver.find_element_by_id('btSign').click() 
            time.sleep(5)
            break
        except:
            with open('error_log.txt', 'a') as f:
                print(datetime.datetime.now(), file=f)
                print('Cannot click clock in buttom', file=f)
            count -= 1
    driver.refresh()
    # Check the status of clock-in
    status, when = check_record('in', driver)
    # Send emails depends on the result
    if status:
        send_email('Clock in successfully')
    else:
        send_email('Unsuccessful clock in')
    driver.quit()

In [9]:
def clock_out(us, ps):
    driver = webdriver.Chrome('C:\\Users\\Administrator\\Downloads\\chromedriver_win32\\chromedriver')
    time.sleep(10)
    driver.get('https://my.ntu.edu.tw/attend/ssi.aspx')
    login(us, ps, driver)
    time.sleep(10)
    try:
        driver.find_element_by_id('btSign2').click()
        time.sleep(10)
    except:
        with open('error_log.txt', 'a') as f:
            print(datetime.datetime.now(), file=f)
            print('Cannot click clock out buttom', file=f)
    driver.refresh()
    status, _ = check_record('out', driver)
    if status:
        send_email('Clock out successfully')
    else:
        send_email('Unsuccessful clock out')
    driver.quit()

In [11]:
# Function that trigger clock-in task.
# I add random delay so that I would not clock-in at the exact same time every day.
def job_Of_Clock_in():
    delay = randint(0,900)
    time.sleep(delay)
    clock_in(us,ps)
    with open('error_log.txt', 'a') as f:
        print('Clock in at', datetime.datetime.today(), file=f)

In [12]:
def job_Of_Clock_out():
    delay = randint(0,900)
    time.sleep(delay)
    clock_out(us,ps)
    with open('error_log.txt', 'a') as f:
        print('Clock out at', datetime.datetime.today(), file=f)

In [13]:
# Using schedule module to schedule clock-in/out tasks
schedule.every().monday.at('08:00').do(job_Of_Clock_in)
schedule.every().monday.at('17:15').do(job_Of_Clock_out)
schedule.every().tuesday.at('08:00').do(job_Of_Clock_in)
schedule.every().tuesday.at('17:15').do(job_Of_Clock_out)
schedule.every().wednesday.at('08:00').do(job_Of_Clock_in)
schedule.every().wednesday.at('17:15').do(job_Of_Clock_out)
schedule.every().thursday.at('08:00').do(job_Of_Clock_in)
schedule.every().thursday.at('17:15').do(job_Of_Clock_out)
schedule.every().friday.at('08:00').do(job_Of_Clock_in)
schedule.every().friday.at('17:15').do(job_Of_Clock_out)

Every 1 week at 17:15:00 do job_Of_Clock_out() (last run: [never], next run: 2020-11-20 17:15:00)

In [ ]:
# Run pending tasks
while True:
    schedule.run_pending()
    time.sleep(1)